In [ ]:
# Code written by Ji Zhang & Liam Ives & Tom Wade, last modified 2024.01.11
# For manuscript "Controllable Multimodal Actuation in Fully Printed Ultrathin Micro-Patterned Electrochemical Actuators"
# email: zhangji1623316718@gmail.com

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from statistics import mean
import glob
from matplotlib.ticker import FormatStrFormatter
from matplotlib.ticker import ScalarFormatter
import scipy as sp
from IPython.display import SVG, display
import warnings
warnings.filterwarnings("ignore")

from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import scipy
from scipy.optimize import curve_fit

def average(lst):
  return sum(lst)/len(lst)
def sd(lst):
  n = len(lst)
  m = sum(lst)/n
  var = sum((np.array(lst)-m)**2)/n
  return np.sqrt(var)

def sds(lst):
  n = len(lst)
  m = sum(lst)/n
  var = sum((np.array(lst)-m)**2)/(n-1)
  return np.sqrt(var)

class my_dictionary(dict):
  def __init__(self):
    self = dict()
  # Function to add key:value
  def add(self, key, value):
    self[key] = value

def linprop(x, k):
    return k*x

from scipy.signal import find_peaks

def peakfinder(x,y,dataframe,height,distance):
  # Funcion adapted from Liam Ives
  xdata = np.array(dataframe[x])
  ydata = np.array(dataframe[y])
  # Find indices of peaks and troughs
  peak_indices, peak_properties = find_peaks(ydata, height=height,distance=distance)
  trough_indices, trough_properties = find_peaks(-ydata, height=-height,distance=distance)  #changed height=-height*0.6
  # Convert peak indices into time values
  peak_times = []
  for peak in peak_indices:
    peak_time = xdata[peak] # find peak time from index given in 'peaks'
    peak_times.append(peak_time)
  trough_times = []
  for trough in trough_indices:
    trough_time = xdata[trough] # find peak time from index given in 'peaks'
    trough_times.append(trough_time)
  print('Finding peak times...')
  print(peak_times)
  print('Finding trough times...')
  print(trough_times)
  return peak_indices, peak_times, trough_indices, trough_times, xdata, ydata

# Plot rc parameters adapted from Tom Wade
fig, ax = plt.subplots() # Need to open a 'plot' so that pcParams actually works
plt.close()
plt.rcParams['text.color'] = 'black'
plt.rcParams['axes.labelcolor'] = 'black'
plt.rcParams['xtick.color'] = 'black'
plt.rcParams['ytick.color'] = 'black'
plt.rcParams['axes.edgecolor'] = 'black'
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['figure.facecolor'] = 'white'

plt.rcParams.update({
    'font.size': 14,          # controls default text sizes
    'axes.labelsize': 14,     # fontsize of the x and y labels
    'axes.titlesize': 14,     # fontsize of the axes title
    'xtick.labelsize': 14,    # fontsize of the tick labels
    'ytick.labelsize': 14,    # fontsize of the tick labels
    'legend.fontsize': 14,    # fontsize of the legend
    'figure.titlesize': 16,   # fontsize of the figure title
})

current_folder_path = os.getcwd().replace('\\', '/')
print(current_folder_path)

In [2]:
def plot_V_I_def(ti, I, V, Q, td, d):
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(4,1, figsize=(20,8), sharex=True, gridspec_kw=dict(height_ratios=[2, 3, 3, 3]))

    ax1.plot(ti, V, color='tab:orange')
    ax1.set_ylabel('$V$/V', color='black')
    ax2.plot(ti, I, color='blue')
    ax2.set_ylabel('$I$/mA', color='black')

    ax1.axhline(y=0, color='#999999', linestyle='--')
    ax2.axhline(y=0, color='#999999', linestyle='--')

    ax3.plot(ti, Q, color='tab:blue')
    ax3.axhline(y=0, color='#999999', linestyle='--')
    ax3.set_ylabel(r'$Q_{fit}$/mC', color='black')

    ax4.plot(td, d, color='green')
    ax4.set(xlabel='Time/s', ylabel=r'$\theta$/rad')
    ax4.xaxis.label.set_color('black')
    ax4.yaxis.label.set_color('black')

    plt.setp(ax1.get_xticklabels(), visible=False)
    plt.setp(ax2.get_xticklabels(), visible=False)

    ax4.axhline(y=d[60], color='#999999', linestyle='--') #or y = d[-1]
    # remove last tick label for the second subplot
    yticks = ax1.yaxis.get_major_ticks()
    yticks[-1].label1.set_visible(False)
    plt.subplots_adjust(hspace=.0)
    yticks = ax2.yaxis.get_major_ticks()
    yticks[-1].label1.set_visible(False)
    plt.subplots_adjust(hspace=.0)
    yticks = ax3.yaxis.get_major_ticks()
    yticks[-1].label1.set_visible(False)
    plt.subplots_adjust(hspace=.0)

    ax1.tick_params(direction='out', length=5, width=1, colors='black')
    ax2.tick_params(direction='out', length=5, width=1, colors='black')
    ax3.tick_params(direction='out', length=5, width=1, colors='black')
    ax4.tick_params(direction='out', length=5, width=1, colors='black')

    # ax1.set_ylim(-0.9,0.9)
    # ax2.set_ylim(-0.007,0.007)
    # ax3.set_ylim(0,0.35)
    # ax2.yaxis.set_major_locator(MultipleLocator(0.01))
    # ax2.yaxis.set_minor_locator(MultipleLocator(0.005))
    # ax3.yaxis.set_major_locator(MultipleLocator(0.1))
    # ax3.yaxis.set_minor_locator(MultipleLocator(0.05))

    ax3.xaxis.set_major_locator(MultipleLocator(50))
    ax3.xaxis.set_minor_locator(MultipleLocator(10))
    ax1.grid(axis = 'x', which='minor')
    ax2.grid(axis = 'x', which='minor')
    ax3.grid(axis = 'x', which='minor')
    ax4.grid(axis = 'x', which='minor')
    ax1.grid(axis = 'x', which='major')
    ax2.grid(axis = 'x', which='major')
    ax3.grid(axis = 'x', which='major')
    ax4.grid(axis = 'x', which='major')
    return fig, (ax1, ax2, ax3, ax4)

In [3]:
## Sweeps

freqs = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
volts = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]

df_tests = {'FreqSweepSq': pd.DataFrame(), 'FreqSweepSin': pd.DataFrame(), 'FreqSweepTri': pd.DataFrame(), 'VoltSweepSq': pd.DataFrame(), 'VoltSweepSin3': pd.DataFrame(), 'VoltSweepTri': pd.DataFrame()}
df_tests['FreqSweepSq'] = pd.DataFrame.from_dict({'freq': [], 'def_pp': [], 'sd_def_pp': []})
df_tests['FreqSweepSin'] = pd.DataFrame.from_dict({'freq': [], 'def_pp': [], 'sd_def_pp': []})
df_tests['FreqSweepTri'] = pd.DataFrame.from_dict({'freq': [], 'def_pp': [], 'sd_def_pp': []})
df_tests['VoltSweepSq'] = pd.DataFrame.from_dict({'volt': [], 'def_pp': [], 'sd_def_pp': []})
df_tests['VoltSweepSin3'] = pd.DataFrame.from_dict({'volt': [], 'def_pp': [], 'sd_def_pp': []})
df_tests['VoltSweepTri'] = pd.DataFrame.from_dict({'volt': [], 'def_pp': [], 'sd_def_pp': []})
df_testsQ = {'FreqSweepSq': pd.DataFrame(), 'FreqSweepSin': pd.DataFrame(), 'FreqSweepTri': pd.DataFrame(), 'VoltSweepSq': pd.DataFrame(), 'VoltSweepSin3': pd.DataFrame(), 'VoltSweepTri': pd.DataFrame()}
df_testsQ['FreqSweepSq'] = pd.DataFrame.from_dict({'freq': [], 'Q_pp': [], 'sd_Q_pp': []})
df_testsQ['FreqSweepSin'] = pd.DataFrame.from_dict({'freq': [], 'Q_pp': [], 'sd_Q_pp': []})
df_testsQ['FreqSweepTri'] = pd.DataFrame.from_dict({'freq': [], 'Q_pp': [], 'sd_Q_pp': []})
df_testsQ['VoltSweepSq'] = pd.DataFrame.from_dict({'volt': [], 'Q_pp': [], 'sd_Q_pp': []})
df_testsQ['VoltSweepSin3'] = pd.DataFrame.from_dict({'volt': [], 'Q_pp': [], 'sd_Q_pp': []})
df_testsQ['VoltSweepTri'] = pd.DataFrame.from_dict({'volt': [], 'Q_pp': [], 'sd_Q_pp': []})

In [4]:
# Open file

test = 'VoltSweepTri' #@param ['FreqSweepSq', 'FreqSweepSin', 'FreqSweepTri', 'VoltSweepSq', 'VoltSweepSin3', 'VoltSweepTri']
                    # Repeat peakfinding for all @params and save results as pickles
fileVI = f'{current_folder_path}/Sample I_Actuation AC_voltage and frequency sweeps/IV/{test}I.csv'
filed = f'{current_folder_path}/Sample I_Actuation AC_voltage and frequency sweeps/def/{test}d.csv'
df_VI_all_1 = pd.read_csv(fileVI, skiprows=1, names=['ti', 'V','I'], encoding='iso-8859-1')
df_VI_all =  df_VI_all_1
# df_VI_all =  df_VI_all_1[df_VI_all_1['ti']<360]
df_d_all_1 = pd.read_csv(filed, skiprows=1, names=['td', 'd'], encoding='iso-8859-1')
df_d_all =  df_d_all_1
# df_d_all =  df_d_all_1[df_d_all_1['td']<360]

In [ ]:
tis = df_VI_all['ti'].to_numpy()
Vs = df_VI_all['V'].to_numpy()
Is = df_VI_all['I'].to_numpy()
tds = df_d_all['td'].to_numpy()
ds = df_d_all['d'].to_numpy()
Qs = scipy.integrate.cumulative_trapezoid(Is, tis, initial=0) # for FreqSweepSin, - 0.032

df_VI_all['Q'] = Qs.tolist()

fig, (ax1, ax2, ax3, ax4) = plot_V_I_def(tis, Is, Vs, Qs, tds, ds)
# ax2.yaxis.set_major_locator(MultipleLocator(0.04))
ax4.set_xlim(0,500)
# ax4.set_xlim(0,360)

# fig.savefig(f'{test}-Qfit.svg',format='svg')

#trapezoidal rule integration may give error?

In [ ]:
# tis, Qs, tds

Qs_new = []

j=0
for i in range(len(tis)):
  if tis[i]>tds[j]:
    Qs_new.append(Qs[i])
    j+=1
    if j<len(tds):
      if tis[i]>tds[j]:
        Qs_new.append(Qs[i])
        j+=1
        if tis[i]>tds[j]:
          Qs_new.append(Qs[i])
          j+=1
          if tis[i]>tds[j]:
            Qs_new.append(Qs[i])
            j+=1
  if len(Qs_new)==len(tds):
    break

plt.plot(tds[:len(Qs_new)], Qs_new, label='QsNew')
plt.plot(tis,Qs)
plt.xlim(0,100)
plt.legend()
print(len(tis), len(tds), len(Qs_new))

# print(tds[:10])
# print(tis[:10])

In [ ]:
# tis, Is, Vs, Qs, tds, ds, Qs_new
fig, ax = plt.subplots(figsize=(6,6))
ax.plot(Qs_new, ds[:len(Qs_new)])
# plt.xlim(-0.05, 0.05)
ax.set_xlabel(r'$Q_{fit}$/mC')
ax.set_ylabel(r'$\theta$/rad')

# plt.text(0,0,r'{test}')
# plt.savefig(f'{test}-Q_theta.svg',format='svg')

In [8]:
if test=='FreqSweepSq':
  marks = [0, 105.2, 170.7, 209.9, 254.2, 295.8, 335.6, 360, 374.7, 389.7, 411.96, 434.8, 448.48, 464.76, 477.84, 493.44, 504.16, 515.52, 525.6, 535.76, 551.48] #for FreqSweepSq
  d0 = 0.2425 #initial deflection
  df_d_all['td_new'] = df_d_all['td'] - 50
elif test=='FreqSweepSin':
  marks = [5, 82, 141.8, 181.9, 224, 260.3, 292, 329.8, 349.8, 366.04, 380, 391.9, 401.8, 412.2, 422.68, 432.76, 443.2, 450.75, 459.28, 474.16, 482.32] #for FreqSweepSin
  d0 = 0.2345
  df_d_all['td_new'] = df_d_all['td'] - 5
elif test=='FreqSweepTri':
  marks = [0, 80.16, 150, 199, 239.2, 270.5, 297.28, 319.5, 342, 363.08, 379.2, 397.6, 412.6, 425.44, 436.4, 451, 462.4, 470.48, 478.72, 488.04, 496.4] #for FreqSweepTri
  d0 = 0.2425
  df_d_all['td_new'] = df_d_all['td'] - 7
elif test=='VoltSweepSq':
  marks = [0, 63.52, 123.24, 183.24, 242.92, 303.16, 363, 422.96, 493.08] #for VoltSweepSq
  d0 = 0.237
elif test=='VoltSweepSin3':
  marks = [5, 73, 134.56, 195.16, 255.48, 305.36, 355.24, 415.44, 489.56] #for VoltSweepSin
  d0 = 0.25
elif test=='VoltSweepTri':
  marks = [16, 64, 124.08, 186.8, 246.44, 297.4, 357.6, 407.44, 466.28] #for VoltSweepTri
  d0 = 0.225
else:
  print('No match found.')

df_VI = my_dictionary()
df_d = my_dictionary()

if test.startswith('Freq'):
  for i in range(len(freqs)):
    values_VI = df_VI_all[(df_VI_all['ti']>marks[i]) & (df_VI_all['ti']<(marks[i+1])+0.05)]
    df_VI.add(freqs[i], values_VI)
    values_d = df_d_all[(df_d_all['td']>marks[i]) & (df_d_all['td']<(marks[i+1])+0.05)]
    df_d.add(freqs[i], values_d)
else:
  for i in range(len(volts)):
    values_VI = df_VI_all[(df_VI_all['ti']>marks[i]) & (df_VI_all['ti']<(marks[i+1])+0.05)]
    df_VI.add(volts[i], values_VI)
    values_d = df_d_all[(df_d_all['td']>marks[i]) & (df_d_all['td']<(marks[i+1])+0.05)]
    df_d.add(volts[i], values_d)

# display(df_VI[1])
# display(df_d[1])

In [ ]:
# Plot graph
# plot parameters adapted from Liam Ives
sns.set(style="ticks", rc={'figure.figsize':(20,2),
                          'lines.linewidth': 2,
                          'font.size': 20,'axes.labelsize': 20,
                          'xtick.labelsize':20,'ytick.labelsize':20,
                          'legend.fontsize':15,'legend.title_fontsize':15,
                          'legend.frameon' : True, 'legend.framealpha': 0.9,
                          'lines.markersize' : 10, 'lines.markeredgewidth' : 3.0
                          }
        ) #figsize is (20,2) but (20, 3) for Sq
plt.figure() # Start a new figure

fig, ax_all = plt.subplots()

i = 0
if test.startswith('Freq'):
  for frequ in freqs:
    color = sns.color_palette()[i-10] if i>9 else sns.color_palette()[i]
    ax_all.plot(df_d[frequ]['td_new'].to_numpy(), df_d[frequ]['d'].to_numpy(), color=color)
    # ax_all.text(0,0.32, f'{frequ} Hz', fontsize=20, color=color) if i==0 else ax_all.text(24*i+18,0.34, f'{frequ}', fontsize=20, color=color)
    i+= 1

if test.startswith('Volt'):
  for volta in volts:
    color = sns.color_palette()[i]
    ax_all.plot(df_d[volta]['td'].to_numpy(), df_d[volta]['d'].to_numpy(), color=color)
    # ax_all.text(20,0.32, f'{volta} V', fontsize=20, color=color) if i==0 else ax_all.text(55*i+38,0.385, f'{volta}', fontsize=20, color=color)
    i+= 1

# ax_all.text(4,0.38, '@\u00B10.5 V', fontsize=20, color='black')
# ax_all.text(4,0.37, '@0.1 Hz', fontsize=20, color='black')

ax_all.axhline(y=d0, color='#999999', linestyle='--')
ax_all.set_xlim(0,500)
ax_all.set_ylim(d0-0.1,d0+0.1)
# ax_all.set_ylim(d0-0.1,d0+0.2)
# ax_all.set_ylim(0.13,0.43) #for freqSweepSq
# ax_all.set_ylim(0.12,0.42) #for voltSweepSq
ax_all.set(xlabel='Time/s', ylabel=r'$\theta_{pp}$/rad')

# ax_all.get_figure().savefig(f'{test}.svg',format='svg')

In [ ]:
# Simplified deflection peakfinding

freq = 0.1 #param [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
T = 1/freq

volt = 0.8 #param [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]

# seg = freq if test.startswith('Freq') else volt

# segs = freqs if test.startswith('Freq') else volts
segs = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6] if test.startswith('Freq') else volts
heightd = d0

# reduced distance to increase no. peaks and troughs recognised
distance_minus = 10

# ignored points when calculating def_pp
front = 1
back = 1

df_tests[test] = df_tests[test].iloc[0:0]
for seg in segs:
  freq = seg if test.startswith('Freq') else 0.1
  #deflection
  distanced = len(df_d[seg]) // ((df_d[seg].iloc[-1]['td'] - df_d[seg].iloc[0]['td'])*freq) - distance_minus
  print(distanced) #no. data pts between two peaks

  peak_def_indices, peak_def_times, trough_def_indices, trough_def_times, xdefdata, ydefdata = peakfinder('td','d',dataframe=df_d[seg],height=heightd, distance = distanced)
  peak_def_values = ydefdata[peak_def_indices]
  trough_def_values = ydefdata[trough_def_indices]

  def_pp = average(peak_def_values[front:(len(peak_def_values)-back)]) - average(trough_def_values[front:(len(peak_def_values)-back)])
  sd_def_pp = np.sqrt(sd(peak_def_values[front:(len(peak_def_values)-back)])**2 + sd(trough_def_values[front:(len(peak_def_values)-back)])**2)

  print(def_pp, sd_def_pp) #peak to peak deflection

  tds = df_d[seg]['td'].to_numpy()
  ds = df_d[seg]['d'].to_numpy()

  ax_deflectiontime = plt.plot(tds,ds,color='red') # Plot deflection data
  ax_def_peaks = plt.plot(peak_def_times, ydefdata[peak_def_indices], 'x',label='Peaks',color='green') # Display peaks on plot
  ax_def_troughs = plt.plot(trough_def_times, ydefdata[trough_def_indices], 'x',label='Troughs',color='plum') # Display troughs on plot
  plt.show()
  df_tests[test].loc[len(df_tests[test])] = [seg, def_pp, sd_def_pp]


display(df_tests)

In [ ]:
# Simplified charge peakfinding

freq = 0.1 #param [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
T = 1/freq

volt = 0.8 #param [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
segs = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6] if test.startswith('Freq') else volts
# seg = freq if test.startswith('Freq') else volt
heightd = d0

# reduced distance to increase no. peaks and troughs recognised
distance_minus = 10

# ignored points when calculating def_pp
front = 1
back = 1

df_testsQ[test] = df_testsQ[test].iloc[0:0]

for seg in segs:
  freq = seg if test.startswith('Freq') else 0.1
  #deflection
  distanced = len(df_VI[seg]) // ((df_VI[seg].iloc[-1]['ti'] - df_VI[seg].iloc[0]['ti'])*freq) - distance_minus
  print(distanced) #no. data pts between two peaks

  peak_def_indices, peak_def_times, trough_def_indices, trough_def_times, xdefdata, ydefdata = peakfinder('ti','Q',dataframe=df_VI[seg],height=0, distance = distanced)
  peak_def_values = ydefdata[peak_def_indices]
  trough_def_values = ydefdata[trough_def_indices]

  Q_pp = average(peak_def_values[front:(len(peak_def_values)-back)]) - average(trough_def_values[front:(len(peak_def_values)-back)])
  sd_Q_pp = np.sqrt(sd(peak_def_values[front:(len(peak_def_values)-back)])**2 + sd(trough_def_values[front:(len(peak_def_values)-back)])**2)

  print(Q_pp, sd_Q_pp) #peak to peak deflection

  tis = df_VI[seg]['ti'].to_numpy()
  Qs = df_VI[seg]['Q'].to_numpy()

  ax_VItime = plt.plot(tis,Qs,color='red') # Plot deflection data
  ax_Q_peaks = plt.plot(peak_def_times, ydefdata[peak_def_indices], 'x',label='Peaks',color='green') # Display peaks on plot
  ax_Q_troughs = plt.plot(trough_def_times, ydefdata[trough_def_indices], 'x',label='Troughs',color='plum') # Display troughs on plot
  plt.show()

  df_testsQ[test].loc[len(df_testsQ[test])] = [seg, Q_pp, sd_Q_pp]

print(df_testsQ)
# df_testsQ[test].to_csv(f'{test}Qpp.csv')

In [100]:
# create folder /Sample I_Actuation AC_voltage and frequency sweeps/output/pickle
pathpickle = f'{current_folder_path}/Sample I_Actuation AC_voltage and frequency sweeps/output/pickle'
df_tests['FreqSweepSq'].to_pickle(f'{pathpickle}/FreqSweepSq')
df_tests['FreqSweepSin'].to_pickle(f'{pathpickle}/FreqSweepSin')
df_tests['FreqSweepTri'].to_pickle(f'{pathpickle}/FreqSweepTri')

In [125]:
pathpickle = f'{current_folder_path}/Sample I_Actuation AC_voltage and frequency sweeps/output/pickle'
df_tests['VoltSweepSq'].to_pickle(f'{pathpickle}/VoltSweepSq')
df_tests['VoltSweepSin3'].to_pickle(f'{pathpickle}/VoltSweepSin3')
df_tests['VoltSweepTri'].to_pickle(f'{pathpickle}/VoltSweepTri')

In [101]:
# create folder /Sample I_Actuation AC_voltage and frequency sweeps/output/pickleQ
pathpickleQ = f'{current_folder_path}/Sample I_Actuation AC_voltage and frequency sweeps/output/pickleQ'
df_testsQ['FreqSweepSq'].to_pickle(f'{pathpickleQ}/FreqSweepSq')
df_testsQ['FreqSweepSin'].to_pickle(f'{pathpickleQ}/FreqSweepSin')
df_testsQ['FreqSweepTri'].to_pickle(f'{pathpickleQ}/FreqSweepTri')

In [126]:
pathpickleQ = f'{current_folder_path}/Sample I_Actuation AC_voltage and frequency sweeps/output/pickleQ'
df_testsQ['VoltSweepSq'].to_pickle(f'{pathpickleQ}/VoltSweepSq')
df_testsQ['VoltSweepSin3'].to_pickle(f'{pathpickleQ}/VoltSweepSin3')
df_testsQ['VoltSweepTri'].to_pickle(f'{pathpickleQ}/VoltSweepTri')

In [127]:
pathpickle = f'{current_folder_path}/Sample I_Actuation AC_voltage and frequency sweeps/output/pickle'
df_tests['FreqSweepSq'] = pd.read_pickle(f'{pathpickle}/FreqSweepSq')
df_tests['FreqSweepSin'] = pd.read_pickle(f'{pathpickle}/FreqSweepSin')
df_tests['FreqSweepTri'] = pd.read_pickle(f'{pathpickle}/FreqSweepTri')
df_tests['VoltSweepSq'] = pd.read_pickle(f'{pathpickle}/VoltSweepSq')
df_tests['VoltSweepSin3'] = pd.read_pickle(f'{pathpickle}/VoltSweepSin3')
df_tests['VoltSweepTri'] = pd.read_pickle(f'{pathpickle}/VoltSweepTri')

In [128]:
pathpickleQ = f'{current_folder_path}/Sample I_Actuation AC_voltage and frequency sweeps/output/pickleQ'
df_testsQ['FreqSweepSq'] = pd.read_pickle(f'{pathpickleQ}/FreqSweepSq')
df_testsQ['FreqSweepSin'] = pd.read_pickle(f'{pathpickleQ}/FreqSweepSin')
df_testsQ['FreqSweepTri'] = pd.read_pickle(f'{pathpickleQ}/FreqSweepTri')
df_testsQ['VoltSweepSq'] = pd.read_pickle(f'{pathpickleQ}/VoltSweepSq')
df_testsQ['VoltSweepSin3'] = pd.read_pickle(f'{pathpickleQ}/VoltSweepSin3')
df_testsQ['VoltSweepTri'] = pd.read_pickle(f'{pathpickleQ}/VoltSweepTri')

# Sweep graphs

In [ ]:
SMALL_SIZE = 12 #8
MEDIUM_SIZE = 12 #10
BIGGER_SIZE = 14 #12

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

fig, ax = plt.subplots()
fig.set_figwidth(5.2)
fig.set_figheight(4)
m_colors = ["blue", "tab:orange", "red", "green"]
x1 = df_tests['FreqSweepSq']['freq'].to_numpy()
y1 = df_tests['FreqSweepSq']['def_pp'].to_numpy()
sd1 = df_tests['FreqSweepSq']['sd_def_pp'].to_numpy()
ax.scatter(x1, y1, marker='s', edgecolors='blue', s=40, facecolors="none", alpha=1, label='Square')
# plt.errorbar(x1, y1, yerr = sd1, fmt='none', color = 'blue')


x2 = df_tests['FreqSweepSin']['freq'].to_numpy()
y2 = df_tests['FreqSweepSin']['def_pp'].to_numpy()
sd2 = df_tests['FreqSweepSin']['sd_def_pp'].to_numpy()
ax.scatter(x2, y2, marker='o', edgecolors='tab:orange', s=40, facecolors="none", alpha=1, label='Sine')
# plt.errorbar(x2, y2, yerr = sd2, fmt='none', color = 'tab:orange')

x3 = df_tests['FreqSweepTri']['freq'].to_numpy()
y3 = df_tests['FreqSweepTri']['def_pp'].to_numpy()
sd3 = df_tests['FreqSweepTri']['sd_def_pp'].to_numpy()
ax.scatter(x3, y3, marker='^', edgecolors='green', s=40, facecolors="none", alpha=1, label='Triangular')
# plt.errorbar(x3, y3, yerr = sd3, fmt='none', color = 'green')

ax.yaxis.set_major_locator(MultipleLocator(0.04))
ax.yaxis.set_minor_locator(MultipleLocator(0.02))

# ax.tick_params(which='both', width=2)
# ax.tick_params(which='major', length=7)
# ax.tick_params(which='minor', length=4)

ax.legend()
# ax.set_ylim(0,0.137)
ax.set_xlabel('Frequency/Hz')
ax.set_ylabel(r'$\theta_{pp}$/rad')
# ax.set_yticks([0.00,0.04,0.08,0.12])
ax.set_xscale('log')
ax.set_yscale('log')
# ax.get_figure().savefig('FreqSweep2.svg',format='svg')

In [ ]:
fig, ax = plt.subplots()
fig.set_figwidth(5.2)
fig.set_figheight(4)
m_colors = ["blue", "tab:orange", "red", "green"]
x11 = df_testsQ['FreqSweepSq']['freq'].to_numpy()
y11 = df_testsQ['FreqSweepSq']['Q_pp'].to_numpy()
sd11 = df_testsQ['FreqSweepSq']['sd_Q_pp'].to_numpy()
ax.scatter(x11, y11, marker='s', edgecolors='blue', s=40, facecolors="none", alpha=1, label='Square')
# plt.errorbar(x11, y11, yerr = sd11, fmt='none', color = 'blue')


x22 = df_testsQ['FreqSweepSin']['freq'].to_numpy()
y22 = df_testsQ['FreqSweepSin']['Q_pp'].to_numpy()
sd22 = df_testsQ['FreqSweepSin']['sd_Q_pp'].to_numpy()
ax.scatter(x22, y22, marker='o', edgecolors='tab:orange', s=40, facecolors="none", alpha=1, label='Sine')
# plt.errorbar(x22, y22, yerr = sd22, fmt='none', color = 'tab:orange')

x33 = df_testsQ['FreqSweepTri']['freq'].to_numpy()
y33 = df_testsQ['FreqSweepTri']['Q_pp'].to_numpy()
sd33 = df_testsQ['FreqSweepTri']['sd_Q_pp'].to_numpy()
ax.scatter(x33, y33, marker='^', edgecolors='green', s=40, facecolors="none", alpha=1, label='Triangular')
# plt.errorbar(x33, y33, yerr = sd33, fmt='none', color = 'green')

ax.yaxis.set_major_locator(MultipleLocator(0.04))
ax.yaxis.set_minor_locator(MultipleLocator(0.02))

# ax.tick_params(which='both', width=2)
# ax.tick_params(which='major', length=7)
# ax.tick_params(which='minor', length=4)

ax.legend()
ax.set_ylim(0,0.08)
ax.set_xlim(0.04,11)
ax.set_xlabel('Frequency/Hz')
ax.set_ylabel(r'$Q_{pp}$/mC')
# ax.set_yticks([0.00,0.04,0.08,0.12])
ax.set_xscale('log')

# ax.get_figure().savefig('FreqSweepQ2.svg',format='svg')

In [ ]:
SMALL_SIZE = 12 #8
MEDIUM_SIZE = 12 #10
BIGGER_SIZE = 14 #12

lw = 1.2 #linewidth

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

fig, ax = plt.subplots()
fig.set_figwidth(5.2)
fig.set_figheight(4)
m_colors = ["blue", "tab:orange", "red", "green"]
x4 = df_tests['VoltSweepSq']['volt'].to_numpy()
y4 = df_tests['VoltSweepSq']['def_pp'].to_numpy()
sd4 = df_tests['VoltSweepSq']['sd_def_pp'].to_numpy()
ax.scatter(x4, y4, marker='s', edgecolors='blue', s=40, facecolors="none", alpha=1, label='Square')
# plt.errorbar(x4, y4, yerr = sd4, fmt='none', color = 'blue')

para4, cov4 = curve_fit(linprop, x4, y4, p0=[0.25])
xs = np.linspace(0,0.8,50)
ys4 = para4[0]*xs
ax.plot(xs,ys4, linewidth=lw, color='blue')

x5 = df_tests['VoltSweepSin3']['volt'].to_numpy()
y5 = df_tests['VoltSweepSin3']['def_pp'].to_numpy()
sd5 = df_tests['VoltSweepSin3']['sd_def_pp'].to_numpy()
ax.scatter(x5, y5, marker='o', edgecolors='tab:orange', s=40, facecolors="none", alpha=1, label='Sine')
# plt.errorbar(x5, y5, yerr = sd5, fmt='none', color = 'tab:orange')

para5, cov5 = curve_fit(linprop, x5, y5, p0=[0.25])
ys5 = para5[0]*xs
ax.plot(xs,ys5, linewidth=lw, color='tab:orange')

x6 = df_tests['VoltSweepTri']['volt'].to_numpy()
y6 = df_tests['VoltSweepTri']['def_pp'].to_numpy()
sd6 = df_tests['VoltSweepTri']['sd_def_pp'].to_numpy()
ax.scatter(x6, y6, marker='^', edgecolors='green', s=40, facecolors="none", alpha=1, label='Triangular')
# plt.errorbar(x6, y6, yerr = sd6, fmt='none', color = 'green')

para6, cov6 = curve_fit(linprop, x6, y6, p0=[0.3])
ys6 = para6[0]*xs
ax.plot(xs,ys6, linewidth=lw, color='green')

ax.yaxis.set_major_locator(MultipleLocator(0.08))
ax.yaxis.set_minor_locator(MultipleLocator(0.04))

ax.legend()
ax.set_ylim(0,0.26)
ax.set_xlim(0,0.84)
ax.set_xlabel('Voltage amplitude/V')
ax.set_ylabel(r'$\theta_{pp}$/rad')

# ax.get_figure().savefig('VoltSweep2.svg',format='svg')

In [ ]:
SMALL_SIZE = 12 #8
MEDIUM_SIZE = 12 #10
BIGGER_SIZE = 14 #12

lw = 1.2 #linewidth

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

fig, ax = plt.subplots()
fig.set_figwidth(5.2)
fig.set_figheight(4)
m_colors = ["blue", "tab:orange", "red", "green"]
x44 = df_testsQ['VoltSweepSq']['volt'].to_numpy()
y44 = df_testsQ['VoltSweepSq']['Q_pp'].to_numpy()
sd44 = df_testsQ['VoltSweepSq']['sd_Q_pp'].to_numpy()
ax.scatter(x44, y44, marker='s', edgecolors='blue', s=40, facecolors="none", alpha=1, label='Square')
# plt.errorbar(x44, y44, yerr = sd44, fmt='none', color = 'blue')

para44, cov44 = curve_fit(linprop, x44, y44, p0=[0.25])
xs = np.linspace(0,0.8,50)
ys44 = para44[0]*xs
ax.plot(xs,ys44, linewidth=lw, color='blue')

x55 = df_testsQ['VoltSweepSin3']['volt'].to_numpy()
y55 = df_testsQ['VoltSweepSin3']['Q_pp'].to_numpy()
sd55 = df_testsQ['VoltSweepSin3']['sd_Q_pp'].to_numpy()
ax.scatter(x55, y55, marker='o', edgecolors='tab:orange', s=40, facecolors="none", alpha=1, label='Sine')
# plt.errorbar(x55, y55, yerr = sd55, fmt='none', color = 'tab:orange')

para55, cov55 = curve_fit(linprop, x55, y55, p0=[0.25])
ys55 = para55[0]*xs
ax.plot(xs,ys55, linewidth=lw, color='tab:orange')

x66 = df_testsQ['VoltSweepTri']['volt'].to_numpy()
y66 = df_testsQ['VoltSweepTri']['Q_pp'].to_numpy()
sd66 = df_testsQ['VoltSweepTri']['sd_Q_pp'].to_numpy()
ax.scatter(x66, y66, marker='^', edgecolors='green', s=40, facecolors="none", alpha=1, label='Triangular')
# plt.errorbar(x66, y66, yerr = sd66, fmt='none', color = 'green')

para66, cov66 = curve_fit(linprop, x66, y66, p0=[0.3])
ys66 = para66[0]*xs
ax.plot(xs,ys66, linewidth=lw, color='green')

ax.yaxis.set_major_locator(MultipleLocator(0.08))
ax.yaxis.set_minor_locator(MultipleLocator(0.04))

ax.legend()
ax.set_ylim(0,0.12)
ax.set_xlim(0,0.84)
ax.set_xlabel('Voltage amplitude/V')
ax.set_ylabel(r'$Q_{pp}$/mC')

# ax.get_figure().savefig('VoltSweepQ2.svg',format='svg')

In [ ]:
fig,ax = plt.subplots()
fig.set_figwidth(5.2)
fig.set_figheight(4)
ax.scatter(y44, y4,marker='s', edgecolors='blue', s=40, facecolors="none", alpha=1, label='Square')
ax.scatter(y55, y5,marker='o', edgecolors='tab:orange', s=40, facecolors="none", alpha=1, label='Sine')
ax.scatter(y66, y6, marker='^', edgecolors='green', s=40, facecolors="none", alpha=1, label='Triangular')
# ax.scatter(y11, y1[:len(y11)],marker='s', edgecolors='tab:blue', s=40, facecolors="none", alpha=1, label='Square_Freq')
# ax.scatter(y22, y2[:len(y22)],marker='o', edgecolors='cyan', s=40, facecolors="none", alpha=1, label='Sine_Freq')
# ax.scatter(y33, y3[:len(y33)], marker='^', edgecolors='purple', s=40, facecolors="none", alpha=1, label='Triangular_Freq')
ax.set_ylim(0, 0.26)
ax.set_xlim(0,0.11)
ax.set_xlabel(r'$Q_{pp}$/mC')
ax.set_ylabel(r'$\theta_{pp}$/rad')
paraa, cova = curve_fit(linprop, np.concatenate((y44,y55,y66)), np.concatenate((y4,y5,y6)), p0=[0.3])
ysa = paraa[0]*xs
print(paraa[0])
ax.legend()
ax.plot(xs,ysa, linewidth=lw, color='grey')
# ax.get_figure().savefig('VoltSweepQ2222.svg',format='svg')

In [ ]:
fig,ax = plt.subplots()
fig.set_figwidth(5.2)
fig.set_figheight(4)
ax.scatter(y11, y1[:len(y11)],marker='s', edgecolors='blue', s=40, facecolors="none", alpha=1, label='Square')
ax.scatter(y22, y2[:len(y22)],marker='o', edgecolors='tab:orange', s=40, facecolors="none", alpha=1, label='Sine')
ax.scatter(y33, y3[:len(y33)], marker='^', edgecolors='green', s=40, facecolors="none", alpha=1, label='Triangular')
ax.set_ylim(0, 0.15)
ax.set_xlim(0,0.08)
ax.set_xlabel(r'$Q_{pp}$/mC')
ax.set_ylabel(r'$\theta_{pp}$/rad')
paraa, cova = curve_fit(linprop, np.concatenate((y11,y22,y33)), np.concatenate((y1[:len(y11)],y2[:len(y22)],y3[:len(y33)])), p0=[0.3])
ysa = paraa[0]*xs
print(paraa[0])
ax.legend()
# ax.plot(xs,ysa, linewidth=lw, color='grey')

# ax.get_figure().savefig('FreqSweepQ2222.svg',format='svg')